In [1]:
import numpy as np
from random import shuffle

In [2]:
import data
import importlib as I
I.reload(data)

<module 'data' from '/home/suriya/_/tf/TF/RNN/toys/binseq/data.py'>

In [3]:
X, Y = data.gen_data()

In [ ]:
X[0].shape

In [4]:
import tensorflow as tf

In [26]:
tf.reset_default_graph()
x_ = [ tf.placeholder(shape=[None,], 
                            dtype=tf.int64, 
                            name='xi_{}'.format(t)) for t in range(20) ]
y_ = [ tf.placeholder(shape=[None,], 
                            dtype=tf.int64, 
                            name='yi_{}'.format(t)) for t in range(20) ]
x_onehot = [ tf.one_hot(x_i_, 2) for x_i_ in x_ ]
init_state = tf.zeros([16, 10])

cell = tf.nn.rnn_cell.BasicRNNCell(10)
rnn_outputs, final_state = tf.nn.rnn(cell, x_onehot, init_state)

# define params
W = tf.get_variable('W', [10, 2])
b = tf.get_variable('b', [2], initializer=tf.constant_initializer(0.0))

# logits
logits = [ tf.matmul(rnn_output, W) + b for rnn_output in rnn_outputs ]
predictions = [ tf.nn.softmax(logit) for logit in logits ]

loss_weights = [ tf.ones_like(y_i_, dtype=tf.float32) for y_i_ in y_ ]
losses = tf.nn.seq2seq.sequence_loss_by_example(rnn_outputs, y_, loss_weights)
loss = tf.reduce_mean(losses)
train_op = tf.train.AdagradOptimizer(learning_rate=0.1).minimize(loss)

In [38]:
tf.nn.rnn?

In [27]:
batchX = X[10:10+16].T
batchY = Y[10:10+16].T

In [31]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_init_state = np.zeros([16, 10])
            
    feed_dict = {x_[t]: batchX[t] for t in range(20)}
    feed_dict.update({y_[t]: batchY[t] for t in range(20)})
    feed_dict.update({init_state : train_init_state})
    
    _, lossv, preds = sess.run([train_op, loss, predictions], feed_dict = feed_dict)
    

In [36]:
preds[0][0]

array([ 0.59172612,  0.40827385], dtype=float32)

In [99]:
import model
import data
import importlib as I
I.reload(model)
I.reload(data)

<module 'data' from '/home/suriya/_/tf/TF/RNN/toys/binseq/data.py'>

In [100]:
net = model.VanillaNet(seqlen=X.shape[-1],
            num_classes = 2,
            state_size = 10,
            batch_size = 128,
            epochs = 10000,
            learning_rate = 0.1,
            ckpt_path='ckpt/')

<log> Building Graph...</log>


In [105]:
train_set = data.rand_batch_gen(X, Y, 128)

In [103]:
sess, i = net.train(train_set, sess, i)


>> Average train loss : 0.27002930292487143

>> Average train loss : 0.16323038280010224

>> Average train loss : 0.1539584795832634

>> Interrupted by user at iteration #3628


In [106]:
a, b = train_set.__next__()

In [107]:
preds = net.predict(sess, a)

In [113]:
a.T[-1], b.T[-1], preds[-1]

(array([0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]))